In [1]:
import pyodbc
import pandas as pd
import numpy as np 

In [2]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=devsppi10db1;'
                      'Port=1433;')

In [3]:
pd.reset_option("display")
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [4]:
#retrieve the sales-call data in HCOId-system level (month level)


sql_stmt = """

/* This query creates a table that compiles media related for a HCO-systemid level promotion response analysis 
Quantity is used to quantify the incremental impact of marketing activity */ 

SELECT 
	r.SystemID, 
	hcoAddr.BaseZip, 
	sh.SystemName, 
	r.YearDesc, 
	r.MonthDesc, 
	--r.BrandName,
	SUM(r.Emails) as Emails, 
	SUM(r.Phones) as Phones, 
	SUM(r.FTF) as FTF, 
	SUM(r.Virtual) as Virtual, 
	SUM(r.Mssg) as Mssg,
	SUM(r.Other) as Other,
	SUM(r.Quantity_adj) as Quantity_adj
/* It is verified that Quantitiy_raw and Quantity_adj have the same value, 
so we can choose either one to represent */ 
FROM 
(
SELECT 
		ISNULL(a.SystemID, b.SystemID) AS SystemID,
		ISNULL(a.YearDesc, b.YearDesc) AS YearDesc,
		ISNULL(a.MonthDesc, b.MonthDesc) AS MonthDesc,
		ISNULL(a.BrandName, b.BrandName) AS BrandName,
		ISNULL(b.Emails, 0) AS Emails,
		ISNULL(b.Phones, 0) AS Phones,
		ISNULL(b.FTF, 0) AS FTF,
		ISNULL(b.Virtual, 0) AS Virtual,
		ISNULL(b.Mssg, 0) AS Mssg,
		ISNULL(b.Other, 0) AS Other,
		ISNULL(a.Quantity_adj, 0) AS Quantity_adj 
FROM (
	SELECT	
		hier.systemid as SystemID, 
		d.YearDesc, 
		d.MonthDesc,
		p.BrandName,
		sum(s.Quantity_adj) AS Quantity_adj	
		FROM (
		SELECT HCOId, SalesTransactionDateId,ProductId, 
		CASE WHEN QuantityAdjusted < 0 THEN 0 ELSE QuantityAdjusted 
		END AS Quantity_adj FROM  [SPECTRUM].[MEAS].[tblFactSalesTransaction] ) s
		INNER JOIN [SPECTRUM].[MEAS].[tblDimDate] d ON s.SalesTransactionDateId = d.DateId
		INNER JOIN	[SPECTRUM].[MEAS].[tblDimProduct] p	ON	s.ProductId = p.ProductId
		INNER JOIN [SPECTRUM].[MEAS].[tblDimHCO] hco ON s.HCOId = hco.HCOId
		INNER JOIN (
		SELECT DISTINCT OutletId, ChildAccountId, ParentAccountId, SystemId FROM [SPECTRUM_ADHOC].[dbo].[tblShyft_HCOHCO_Hierarchy] ) hier 
		ON hco.HCOId = hier.OutletId
		WHERE s.HCOId <> -1 AND p.BrandName = 'ROLVEDON' 
		GROUP BY  
				d.YearDesc, 
				d.MonthDesc,
				p.BrandName, 
				hier.systemid
) a
/* ProductId =1 (Rolvedon). All ProductId in FasctSalesTrasaction equals to 1 
 FactSalesTransaction (hco-date level) has 2819 rows and
 a (hco-month level) has 960 rows */

FULL OUTER JOIN (
SELECT 
	fc.Systemid as SystemID,
    d.YearDesc,
    d.MonthDesc,
    p.BrandName,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Email_vod' THEN fc.CallId END) AS Emails,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Phone_vod' THEN fc.CallId END) AS Phones,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Face_to_face_vod' THEN fc.CallId END) AS FTF,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Video_vod' THEN fc.CallId END) AS Virtual,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Message_vod' THEN fc.CallId END) AS Mssg,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Other_vod' THEN fc.CallId END) AS Other
FROM
    (
        SELECT  hco.HCOSourceID, hco.AccountType, hier.Systemid, fc1.*  FROM [SPECTRUM].[MEAS].[tblFactCall]  fc1
		INNER join [SPECTRUM].[MEAS].[tblDimHCO] hco
		on fc1.HCOId = hco.HCOId AND AccountType = 'Child Account'  
		INNER JOIN
		(SELECT DISTINCT  sh. ChildAccountID, sh.systemid 
		FROM [SPECTRUM_ADHOC].[dbo].[tblShyft_HCOHCO_Hierarchy] sh) hier ON hco.HCOId = hier.ChildAccountID 
		WHERE fc1.HCOId <> -1
		/* 
		There are 4280 rows/ 1063 distinct HCOId in tblFactCall that has AccoutType as  'Child Account' 
		There are 296 rows that the HCOId has no corresponding systemid. 
		The total amount of row is 3984  */
		UNION ALL 

		SELECT   hco.HCOSourceID, hco.AccountType, hier.Systemid, fc2.*  FROM [SPECTRUM].[MEAS].[tblFactCall]  fc2
		INNER join [SPECTRUM].[MEAS].[tblDimHCO] hco
		on fc2.HCOId = hco.HCOId AND AccountType = 'Parent Account'  
		INNER JOIN
		(SELECT DISTINCT  sh.ParentAccountID , sh.systemid 
		FROM [SPECTRUM_ADHOC].[dbo].[tblShyft_HCOHCO_Hierarchy] sh) hier ON hco.HCOId = hier.ParentAccountID 
		WHERE fc2.HCOId <> -1 AND fc2.HCOId NOT IN 
		(select  ParentAccountID
		from [SPECTRUM_ADHOC].[dbo].[tblShyft_HCOHCO_Hierarchy]
		GROUP BY ParentAccountID
		HAVING  count(distinct Systemid) > 1) 
		/* There are 215 rows in tblFactCall that has AccoutType as  'Parent Account', 
		18 of them are in the list (There are 8 ParentAccount that can correspond to two systemid)
		hence, there are 36 rows having no systemid. 
		The total amount of rows: 173 */ 

		UNION ALL 
		SELECT  hco.HCOSourceID, hco.AccountType , hco.HCOId as Systemid, fc3.*  FROM [SPECTRUM].[MEAS].[tblFactCall]  fc3
		INNER join [SPECTRUM].[MEAS].[tblDimHCO] hco
		on fc3.HCOId = hco.HCOId AND AccountType = 'system'  
		/* There are 2001 rows in tblFactCall that HCOId has AccoutType as  'System'
		There are 9403 rows/ 9 distinct HCOId has AccoutType as  'NULL' */  		   
    )  fc
/* tblFactCall has 15899 rows. 
After rollup the table to systemid-HCOId level, there are 6158 rows in total */ 
INNER JOIN
    [SPECTRUM].[dbo].[tblDimCallChannel] cc ON fc.CallChannelId = cc.CallChannelId
INNER JOIN
    [SPECTRUM].[MEAS].[tblDimDate] d ON fc.CallDateId = d.DateId
INNER JOIN
    [SPECTRUM].[MEAS].[tblFactCallDetail] fcd ON fc.CallSourceId = fcd.CallSourceId
LEFT JOIN
    [SPECTRUM].[MEAS].[tblDimProduct] p ON fcd.ProductId = p.ProductId
GROUP BY
    fc.systemid,
    d.YearDesc,
    d.MonthDesc,
    p.BrandName
) b 
/*FactCall has 13858 (date-level) rows and b (month level) has 2885 rows*/
ON a.SystemID = b.SystemID AND a.YearDesc = b.YearDesc
	AND a.MonthDesc = b.MonthDesc
	AND a.BrandName = b.BrandName
	) r 
INNER JOIN [SPECTRUM].[MEAS].[tblDimHCO] hco ON r.SystemID = hco.HCOId
LEFT OUTER JOIN (SELECT DISTINCT Systemid, SystemName FROM [SPECTRUM_ADHOC].[dbo].[tblShyft_HCOHCO_Hierarchy]) sh ON sh.Systemid = r.SystemID
LEFT OUTER JOIN [SPECTRUM].[MEAS].[tblDimEntityAddress] hcoEntAddr
	ON hco.HCOId = hcoEntAddr.EntityId 
	AND hcoEntAddr.EntityType = 'HCO'
	AND hcoEntAddr.Active = 1
	AND hcoEntAddr.IsPrimary = 1
LEFT OUTER JOIN [SPECTRUM].[MEAS].[tblDimAddress] hcoAddr
	ON hcoEntAddr.AddressId = hcoAddr.AddressId
	AND hcoAddr.Active = 1

GROUP BY 
	r.SystemID, 
	sh.SystemName,
	hcoAddr.BaseZip,
    r.YearDesc,
    r.MonthDesc,
    r.BrandName
ORDER BY r.YearDesc DESC, r.MonthDesc DESC

"""

salescall = pd.read_sql(sql_stmt, conn)

In [5]:
salescall.head(5)

,SystemID,BaseZip,SystemName,YearDesc,MonthDesc,Emails,Phones,FTF,Virtual,Mssg,Other,Quantity_adj
0,4769,29732,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",2023,May,0,1,0,0,0,0,0.0
1,5327,33709,"COMPREHENSIVE HEMATOLOGY ONCOLOGY, LLC",2023,May,0,0,1,0,0,0,0.0
2,5328,92543,HEMATOLOGY ONCOLOGY CONSULTANTS,2023,May,0,0,0,0,0,0,64.0
3,5414,19013,"ASSOCIATES IN HEMATOLOGY AND ONCOLOGY, PC",2023,May,0,0,1,0,0,0,1.0
4,5456,93940,PACIFIC CANCER CARE,2023,May,1,0,0,0,0,0,0.0


In [6]:
#retrieve the OK data in HCOId-systemid level (FNCL, bus, bed) 


sql_stmt = """

--OK table prep (FNCL, bus, bed) 
--IQVIA OneKey table prep (FNCL, bus, bed) 
SELECT --hco.HCOId,
	   hier.systemid as SystemID,
	--   fncl column name
	--   fncl.DATA_YR_STRT_DT, fncl.DATA_YR_END_DT, 
	   fncl.DDS_IN_PD_CNT, fncl.INPAT_REV_AMT, 
	   fncl.OUTPAT_REV_AMT, fncl.TTL_PAT_REV_AMT, fncl.NET_PAT_REV_AMT, fncl.NON_PAT_REV_AMT, 
	   fncl.GOVT_REV_AMT, fncl.HSP_IN_REV_AMT, fncl.HSP_OUT_REV_AMT, fncl.OPRG_EXP_AMT, 
	   fncl.OTHR_EXP_AMT, fncl.TTL_EXP_AMT, fncl.SUPP_COST_AMT, fncl.DRG_COST_AMT, 
	   fncl.COST_PER_BED_AMT, fncl.NET_INCM_AMT, fncl.NET_INCM_PAT_AMT, 
	   fncl.OTHR_INCM_AMT, fncl.TTL_ASSETS_AMT, fncl.FIX_ASSETS_AMT, 
	   fncl.CUR_ASSETS_AMT, fncl.OTHR_ASSETS_AMT, fncl.TTL_LIAB_AMT, 
	   fncl.CUR_LIAB_AMT, fncl.LNG_TERM_LIAB_AMT, fncl.FUND_BAL_AMT, 
	   fncl.FTE_EMP_CNT, fncl.TTL_SAL_AMT, fncl.FRNG_BEN_AMT,
	 --  hco_bed_total column name
	 --  hco_bed_total.BED_ID, hco_bed_total.BED_CD, hco_bed_total.BED_DESC, 
	   hco_bed_total.LIC_BED_CNT, 
	   hco_bed_total.CENS_BED_CNT, hco_bed_total.STFD_BED_CNT, 
	  
	--   hco_bus column name
	   hco_bus.avgoccrate, hco_bus.routineday, hco_bus.H_pdays, hco_bus.avglos, 
	   hco_bus.mparmix, hco_bus.mcaredrg, hco_bus.ttldrg, hco_bus.MCR_alos, 
	   hco_bus.hdrg, hco_bus.MCR_pdays
	   FROM [SPECTRUM].[MEAS].[tblDimHCO] hco

INNER JOIN 
(SELECT TPSEntityId as HCOSourceId_fncl, iq_hco_fncl.*
FROM [SPECTRUM_ADHOC].[dbo].[tblDataFeed_IQVIA_OneKey_HCO_FNCL_FACT] iq_hco_fncl
--tblDataFeed_IQVIA_OneKey_HCO_FNCL_FACT has 4149 rows
INNER JOIN (
    SELECT *
    FROM [SPECTRUM_CM].[CM].[tblOutletAlternateId]
    WHERE TPSDataFeedId = 6
) oa ON iq_hco_fncl.HCO_HCE_ID = oa.DataProviderUniqueIdentifier COLLATE SQL_Latin1_General_CP1_CI_AS
-- "SQL_Latin1_General_CP1_CI_AS" indicates to read collumn via "class insensitive"
  ) fncl ON hco.HCOSourceID = fncl.HCOSourceId_fncl
 --fncl has shape: 4149x33

LEFT OUTER JOIN 
(SELECT TPSEntityId as HCOSourceId_bed, bed.*
FROM [SPECTRUM_ADHOC].[dbo].[tblDataFeed_IQVIA_OneKey_HCO_BED_FACT] bed 
INNER JOIN (
    SELECT *
    FROM [SPECTRUM_CM].[CM].[tblOutletAlternateId]
    WHERE TPSDataFeedId = 6
) oa ON bed.HCO_HCE_ID = oa.DataProviderUniqueIdentifier COLLATE SQL_Latin1_General_CP1_CI_AS
WHERE BED_DESC = 'Total' 
  ) hco_bed_total ON hco.HCOSourceID = hco_bed_total.HCOSourceId_bed 
 --hco_bed_total has shape: 4725 x 8 

LEFT OUTER JOIN 
(SELECT TPSEntityId as HCOSourceId_bus, bus.* FROM (SELECT
    HCO_HCE_ID,
    MAX(CASE WHEN DETL_CD = 'avgoccrate' THEN DETL_NBR END) AS avgoccrate,
    MAX(CASE WHEN DETL_CD = 'routineday' THEN DETL_NBR END) AS routineday,
    MAX(CASE WHEN DETL_CD = 'H_pdays' THEN DETL_NBR END) AS H_pdays,
    MAX(CASE WHEN DETL_CD = 'avglos' THEN DETL_NBR END) AS avglos,
    MAX(CASE WHEN DETL_CD = 'mparmix' THEN DETL_NBR END) AS mparmix,
    MAX(CASE WHEN DETL_CD = 'mcaredrg' THEN DETL_NBR END) AS mcaredrg,
    MAX(CASE WHEN DETL_CD = 'ttldrg' THEN DETL_NBR END) AS ttldrg,
    MAX(CASE WHEN DETL_CD = 'MCR_alos' THEN DETL_NBR END) AS MCR_alos,
    MAX(CASE WHEN DETL_CD = 'hdrg' THEN DETL_NBR END) AS hdrg,
    MAX(CASE WHEN DETL_CD = 'MCR_pdays' THEN DETL_NBR END) AS MCR_pdays
FROM
    [SPECTRUM_ADHOC].[dbo].[tblDataFeed_IQVIA_OneKey_HCO_BUS_DETL_FACT]
GROUP BY
    HCO_HCE_ID
HAVING
    COALESCE(MAX(CASE WHEN DETL_CD = 'avgoccrate' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'routineday' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'H_pdays' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'avglos' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'mparmix' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'mcaredrg' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'ttldrg' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'MCR_alos' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'hdrg' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'MCR_pdays' THEN DETL_NBR END)) IS NOT NULL
) bus
INNER JOIN (
    SELECT *
    FROM [SPECTRUM_CM].[CM].[tblOutletAlternateId]
    WHERE TPSDataFeedId = 6
) oa ON bus.HCO_HCE_ID = oa.DataProviderUniqueIdentifier COLLATE SQL_Latin1_General_CP1_CI_AS
) hco_bus ON hco.HCOSourceID = hco_bus.HCOSourceId_bus 
 --hco_bus has shape: 4145x12 

INNER JOIN
(SELECT DISTINCT  sh. ChildAccountID, sh.systemid 
FROM [SPECTRUM_ADHOC].[dbo].[tblShyft_HCOHCO_Hierarchy] sh) hier ON hco.HCOId = hier.ChildAccountID 
/* before the inner join, there are 4149 rows of OK data, 
after performing the inner join, there are 4101 rows.
There are 48 HCOId in the OK data that have no corresponding SystemId  */ 

"""
onekey = pd.read_sql(sql_stmt, conn)
onekey.head(5)

,SystemID,DDS_IN_PD_CNT,INPAT_REV_AMT,OUTPAT_REV_AMT,TTL_PAT_REV_AMT,NET_PAT_REV_AMT,NON_PAT_REV_AMT,GOVT_REV_AMT,HSP_IN_REV_AMT,HSP_OUT_REV_AMT,...,avgoccrate,routineday,H_pdays,avglos,mparmix,mcaredrg,ttldrg,MCR_alos,hdrg,MCR_pdays
0,28883,364,543298,742860,1286158,238973,6557,8,45654,40720,...,.4694,28689.0000,28689.0000,3.4762,.3763,3073.0000,8253.0000,4.0000,8253.0000,11668.0000
1,24297,364,648424,566283,1214708,357490,34701,0,122031,56631,...,.4968,46000.0000,46000.0000,3.8937,.3064,3242.0000,11814.0000,5.0000,11814.0000,15170.0000
2,23700,364,1809286,1326432,3135717,1190197,11677,0,274011,169374,...,.4560,84789.0000,84789.0000,3.2531,.2779,5185.0000,26064.0000,4.0000,26064.0000,19331.0000
3,28692,364,1764767,1699644,3464411,621114,1749,0,267288,194350,...,.6330,80166.0000,80166.0000,3.6980,.2294,4429.0000,21678.0000,4.0000,21678.0000,15761.0000
4,28768,364,1649333,2093668,3743001,1183699,257870,0,239585,2026152,...,.5631,110051.0000,110051.0000,5.4984,.3229,4115.0000,20015.0000,5.0000,20015.0000,22566.0000


In [27]:
#sum over the non-media variables in the OneKey data grouping by systemid 
# List of columns to sum
columns_ok = onekey.columns[1:]

# Sum the values based on SystemID
onekey[columns_ok] = onekey[columns_ok].astype(float)
onekey = onekey.groupby('SystemID',as_index = False )[columns_ok].sum()

In [28]:
import pandas as pd
import numpy as np

class DataBalancer:
    def __init__(self, data_df):
        self.df = data_df

    def datapreprocess(self):
        #  create a standardized Date column
        # Convert 'Month' and 'Year' columns to datetime format
        self.df['Date'] = pd.to_datetime(self.df['MonthDesc'] + ' ' + self.df['YearDesc'].astype(str), format='%B %Y')
        # Convert 'Date' column to the desired format 'yyyy-mm'
        self.df['Date'] = self.df['Date'].dt.strftime('%Y-%m')
        # Drop unnecessary columns: 'YearDesc' and 'MonthDesc'
        self.df.drop(['MonthDesc', 'YearDesc'], axis=1, inplace=True)
        return self.df


    def balance_data(self, df: pd.DataFrame, npi:str, date: str, seg_cols = None):
        #npi : HCOId, HCPId, or SourceId ,...etc. 
        #date: state the column name for Date 
        if seg_cols is None:
            seg_cols = []
        #Create a dataframe that has all combination of HCOId, Date 
        hco_ids = self.df[npi].unique() 
        dates = pd.date_range(start= self.df[date].min() , end= self.df[date].max() , freq='M')
        index = pd.MultiIndex.from_product([hco_ids, dates], names=[npi, date])
        balanced_df = pd.DataFrame(index=index).reset_index()
        # Convert 'Date' column to the desired format 'yyyy-mm'
        balanced_df[date] = balanced_df[date].dt.strftime('%Y-%m')
        balanced_df[date] = balanced_df[date].astype(str)
        self.df[date] = self.df[date].astype(str)

        df_out = balanced_df.merge(self.df, on=[npi, date], how='left')

         # if static hcp level columns exist, fill the column with original values
        if seg_cols is not None:
            for i in seg_cols:
                # can try to use apply fill na with non-na value groupby
                df_out[i] = df_out.groupby([npi])[i].bfill()
                df_out[i] = df_out.groupby([npi])[i].ffill()
            #considering there are
            df_out[seg_cols] = df_out[seg_cols].fillna('null') 
        return df_out.fillna(0)

def winsorize(df: pd.DataFrame, cols: [], percentile: int) -> pd.DataFrame:
    df_out = df.copy()
    for i in cols:
        threshold = np.percentile(df_out[i], percentile)
        df_out[i] = np.where(df_out[i] > threshold, threshold, df_out[i])
    return df_out


In [29]:
# Create an instance of DataBalancer 
balancer = DataBalancer(data_df=salescall)

# Call the datapreprocess method and pass the Zip column name as an argument
# The method will preprocess the data and return the updated DataFrame
processed_df = balancer.datapreprocess()
df_out = balancer.balance_data(processed_df, npi = 'SystemID', date = 'Date', seg_cols = ['BaseZip','SystemName'])


In [30]:
df_out.head(5)

,SystemID,Date,BaseZip,SystemName,Emails,Phones,FTF,Virtual,Mssg,Other,Quantity_adj
0,4769,2022-10,29732,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4769,2022-11,29732,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,4769,2022-12,29732,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,1.0,1.0,0.0,0.0,0.0,50.0
3,4769,2023-01,29732,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",2.0,2.0,0.0,0.0,0.0,0.0,0.0
4,4769,2023-02,29732,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",1.0,2.0,0.0,0.0,0.0,0.0,0.0


In [31]:
npi = 'SystemID'
df_all = df_out.merge(onekey, on=[npi], how = 'left')
df_all.to_csv("spectrum_balanced_raw.csv",index=False) 

In [32]:
#Preprocessing
# Fill NaN values with the mean of each column
for column in columns_ok:
    mean_value = round(df_all[column].mean(),2)
    df_all[column].fillna(mean_value, inplace=True)

#winsorize on all media variables and quantity
column_win = ['Emails','Phones','FTF','Virtual', 'Mssg', 'Other', 'Quantity_adj']
df_all_win = winsorize(df= df_all, cols =  column_win, percentile= 99.9 )


In [33]:
df_all.to_csv("spectrum_balanced.csv",index=False)
df_all_win.to_csv("spectrum_balanced_win.csv",index=False)

In [34]:
df_all_win.head(5)

,SystemID,Date,BaseZip,SystemName,Emails,Phones,FTF,Virtual,Mssg,Other,...,avgoccrate,routineday,H_pdays,avglos,mparmix,mcaredrg,ttldrg,MCR_alos,hdrg,MCR_pdays
0,4769,2022-10,29732,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
1,4769,2022-11,29732,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,0.0,1.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
2,4769,2022-12,29732,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,1.0,1.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
3,4769,2023-01,29732,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",2.0,2.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
4,4769,2023-02,29732,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",1.0,2.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8


In [35]:
df_all_win.corr()

,SystemID,Emails,Phones,FTF,Virtual,Mssg,Other,Quantity_adj,DDS_IN_PD_CNT,INPAT_REV_AMT,...,avgoccrate,routineday,H_pdays,avglos,mparmix,mcaredrg,ttldrg,MCR_alos,hdrg,MCR_pdays
SystemID,1.000000,0.037912,-0.004015,-0.008886,0.019179,0.029247,0.000058,0.000115,-0.009650,-0.001739,...,-0.012684,-0.004847,-0.004847,-0.011932,-0.010724,-0.005917,-0.006421,-0.012221,-0.006421,-0.005495
Emails,0.037912,1.000000,0.387262,0.358599,0.282910,0.074155,0.034682,0.171971,0.062356,0.033459,...,0.057246,0.043535,0.043535,0.034820,0.060852,0.049422,0.043708,0.046747,0.043708,0.048695
Phones,-0.004015,0.387262,1.000000,0.317963,0.171426,0.062886,0.003224,0.102287,0.058069,0.019364,...,0.045565,0.029868,0.029868,0.037429,0.064174,0.042203,0.031146,0.049914,0.031146,0.040304
FTF,-0.008886,0.358599,0.317963,1.000000,0.114316,0.067133,-0.009802,0.201242,0.043905,0.018110,...,0.039609,0.028309,0.028309,0.024341,0.043222,0.034654,0.028322,0.034600,0.028322,0.034354
Virtual,0.019179,0.282910,0.171426,0.114316,1.000000,0.042546,0.028822,0.046029,0.044651,0.023172,...,0.038902,0.034600,0.034600,0.020201,0.042456,0.039233,0.032914,0.027375,0.032914,0.038315
Mssg,0.029247,0.074155,0.062886,0.067133,0.042546,1.000000,0.000308,0.022417,0.004438,0.001139,...,0.004885,-0.000069,-0.000069,-0.000307,0.004494,0.001635,0.001168,0.000426,0.001168,0.000288
Other,0.000058,0.034682,0.003224,-0.009802,0.028822,0.000308,1.000000,-0.018944,0.140098,0.113876,...,0.135249,0.119077,0.119077,0.072379,0.135346,0.129389,0.124849,0.094504,0.124849,0.122695
Quantity_adj,0.000115,0.171971,0.102287,0.201242,0.046029,0.022417,-0.018944,1.000000,-0.000180,0.000533,...,-0.000137,0.000373,0.000373,-0.000236,-0.000222,0.000155,0.000258,-0.000224,0.000258,0.000315
DDS_IN_PD_CNT,-0.009650,0.062356,0.058069,0.043905,0.044651,0.004438,0.140098,-0.000180,1.000000,0.770817,...,0.973564,0.862991,0.862991,0.715139,0.972840,0.885615,0.863585,0.856246,0.863585,0.882595
INPAT_REV_AMT,-0.001739,0.033459,0.019364,0.018110,0.023172,0.001139,0.113876,0.000533,0.770817,1.000000,...,0.846443,0.947945,0.947945,0.436042,0.664474,0.926990,0.952306,0.564748,0.952306,0.919851


In [36]:
# Calculate the correlation matrix
correlation_matrix = df_all_win.corr()

# Get the correlation values with 'Quantity_adj' column
correlation_with_quantity = correlation_matrix['Quantity_adj']

# Exclude 'Quantity_adj' itself from the calculation
correlation_with_quantity = correlation_with_quantity.drop('Quantity_adj')

# Find the factor with the highest correlation
highest_correlation_factor = correlation_with_quantity.idxmax()
highest_correlation_value = correlation_with_quantity.max()


# Find the top 4 factors with the highest correlation
top_correlation_factors = correlation_with_quantity.abs().nlargest(3).index
top_correlation_values = correlation_with_quantity[top_correlation_factors]

print("Top 3 factors with the highest correlation with 'Quantity_adj':")
for factor, value in zip(top_correlation_factors, top_correlation_values):
    print(f"Factor: {factor}, Correlation: {value}")

print('\n')

print(f"The factor with the highest correlation with 'Quantity_adj' is: {highest_correlation_factor}")
print(f"The highest correlation value is: {highest_correlation_value}")

Top 3 factors with the highest correlation with 'Quantity_adj':
Factor: FTF, Correlation: 0.20124216824770338
Factor: Emails, Correlation: 0.17197071330876923
Factor: Phones, Correlation: 0.10228676257627772


The factor with the highest correlation with 'Quantity_adj' is: FTF
The highest correlation value is: 0.20124216824770338


In [37]:
# Calculate the correlation matrix
correlation_matrix = df_all_win.corr()
correlation_matrix['Quantity_adj']

SystemID             0.000115
Emails               0.171971
Phones               0.102287
FTF                  0.201242
Virtual              0.046029
Mssg                 0.022417
Other               -0.018944
Quantity_adj         1.000000
DDS_IN_PD_CNT       -0.000180
INPAT_REV_AMT        0.000533
OUTPAT_REV_AMT       0.000151
TTL_PAT_REV_AMT      0.000379
NET_PAT_REV_AMT      0.000731
NON_PAT_REV_AMT      0.000707
GOVT_REV_AMT         0.000276
HSP_IN_REV_AMT       0.001845
HSP_OUT_REV_AMT     -0.000203
OPRG_EXP_AMT         0.000555
OTHR_EXP_AMT        -0.000260
TTL_EXP_AMT          0.000547
SUPP_COST_AMT       -0.000224
DRG_COST_AMT        -0.000073
COST_PER_BED_AMT    -0.000249
NET_INCM_AMT         0.001763
NET_INCM_PAT_AMT     0.002091
OTHR_INCM_AMT       -0.000135
TTL_ASSETS_AMT       0.000275
FIX_ASSETS_AMT       0.000064
CUR_ASSETS_AMT       0.001087
OTHR_ASSETS_AMT      0.000003
TTL_LIAB_AMT         0.000270
CUR_LIAB_AMT        -0.000002
LNG_TERM_LIAB_AMT    0.000427
FUND_BAL_A

In [39]:
# Check if the minimum date for every SystemID is '2022-10-01'

# Assuming the date column is named "Date" and SystemID column is named "SystemID"
# First, convert the "Date" column to datetime if it's not already in datetime format
df_all_win['Date'] = pd.to_datetime(df_all_win['Date'])

# Group by SystemID and calculate the minimum and maximum date for each group
date_min_max = df_all.groupby('SystemID')['Date'].agg(['min', 'max'])
is_min_2022_10_01 = (date_min_max['min'] == '2022-10').all()

# Check if the maximum date for every SystemID is '2023-07-01'
is_max_2023_07_01 = (date_min_max['max'] == '2023-07').all()

print("Is the minimum date for every SystemID 2022-10?", is_min_2022_10_01)
print("Is the maximum date for every SystemID 2023-07?", is_max_2023_07_01)

Is the minimum date for every SystemID 2022-10? True
Is the maximum date for every SystemID 2023-07? True


In [42]:
#Find the systemid that is below the start point in the ? channel 
# Filter SystemID with average FTF < 1 for each month
start_point = 0.6
filtered_systems = df_all_win.groupby(['SystemID'])['FTF'].mean() < start_point
system_ids = filtered_systems[filtered_systems].index.get_level_values('SystemID').unique().tolist()

print("The first 5 SystemIDs with average ftf < 0.6 for each month:", system_ids[:5])
print("The amount of systemid that are below the starting point:" , len(system_ids))
print("The number of unique SystemId in the dataset: " , df_all_win['SystemID'].nunique())

The first 5 SystemIDs with average ftf < 0.6 for each month: [4746, 4769, 4777, 4905, 5285]
The amount of systemid that are below the starting point: 843
The number of unique SystemId in the dataset:  878


In [43]:
#retrieve all the HCO name 

sql_stmt = """
SELECT SystemID, SystemName FROM [SPECTRUM_ADHOC].[dbo].[tblShyft_HCOHCO_Hierarchy]
"""
systemname = pd.read_sql(sql_stmt, conn)
system_names = systemname.loc[systemname['SystemID'].isin(system_ids), 'SystemName'].tolist()
system_names[:5]


['THE QUEENS HEALTH SYSTEMS',
 'CHRISTUS HEALTH',
 'UNIVERSITY OF CALIFORNIA HEALTH',
 'PENN MEDICINE',
 'UNITEDHEALTH GROUP']